In [1]:
%matplotlib inline

import pickle
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import structure

In [2]:
##################################
# if you want to load the object #
##################################

try:
    PATH = '/Users/maxwellclarke/Documents/data/fma_metadata/segmented'
    os.chdir(PATH)
except:
    PATH = r'C:\Users\james\Documents\data\fma_metadata\segmented'
    os.chdir(PATH)
    
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)
    
dfs = data.segmented_dfs # already sorted data

In [3]:
data.tracks.track.groupby('genre_top').count().sort_values('title', ascending=False)['title']

genre_top
Rock                   14182
Experimental           10608
Electronic              9371
Hip-Hop                 3552
Folk                    2803
Pop                     2332
Instrumental            2079
International           1389
Classical               1230
Jazz                     571
Old-Time / Historic      554
Spoken                   423
Country                  194
Soul-RnB                 175
Blues                    110
Easy Listening            24
Name: title, dtype: int64

### Picking all features regarding mean or standard deviation

In [4]:
import re

r = re.compile('.*mean')
q = re.compile('.*std')

to_concat = [df for key, df in dfs['features'].items() if (bool(r.match(key)) | bool(q.match(key)))] # means and stds dfs

df = pd.concat(to_concat +[data.tracks.track[['genre_top']]], axis=1, join='inner')

mask = df['genre_top'].isin(['Rock', 'Electronic', 'Hip-Hop', 'Folk', 'Classical'])

print(df.shape)
df = df[mask]
print(df.shape)


(91214, 149)
(25019, 149)


In [5]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split


X = df.drop('genre_top', axis=1)
y = df['genre_top']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.2, stratify=y)

In [6]:
from sklearn.linear_model import LogisticRegression
from copy import deepcopy
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score



def multi_class_test(X, y, est):
    """Cross validates and evaluates multi-class model."""
    genres = set(y)
    X = X.values
    y = y.values
    
    classes = set(y)
    
    skf = StratifiedKFold(n_splits=5, random_state=42)
    
    scores = {
        'train': [],
        'test': []
    }
    
    estimators = []
    for tr_ix, te_ix in skf.split(X, y):
        esti = deepcopy(est)

        X_train, X_test = X[tr_ix], X[te_ix]
        y_train, y_test = y[tr_ix], y[te_ix]

        esti.fit(X_train, y_train)
        
        y_train_preds = esti.predict(X_train)
        y_test_preds = esti.predict(X_test)
        y_preds_proba_test = esti.predict_proba(X_test)
        y_preds_proba_train = esti.predict_proba(X_train)

        scores['train'].append(accuracy_score(y_train, y_train_preds))
        scores['test'].append(accuracy_score(y_test, y_test_preds))
            
        estimators.append(esti)
    
    return scores, estimators

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA

pipe = Pipeline([
    ('std_scl', StandardScaler()),
    ('lr', LogisticRegression())
])

param_grid = [
    {'lr__C': [.01, .1, 1, 2,5]}
]

gcv = GridSearchCV(pipe, param_grid=param_grid, verbose=0)

In [16]:
gcv.fit(X_train, y_train)


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:460: FutureWar

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('std_scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'lr__C': [0.01, 0.1, 1, 2, 5]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
kpca = KernelPCA()
kpca.fit(X_train.iloc[100, :7])

NameError: name 'y_test' is not defined